# Practice Case - Statistics

## Scenario
The data-set contains aggregate individual statistics for 67 NBA seasons. from basic box-score
attributes such as points, assists, rebounds etc., to more advanced money-ball like features such as
Value Over Replacement.

## Project Goals
1. Who is the youngest and oldest player in the NBA in 2017 for each team (Tm) ?
2. Which player has the most minutes played (MP) in each position (Pos)?
3. Which team has the highest average total rebound percentage (TRB%), assist percentage (AST%), steal percentage (STL%), and block percentage (BLK%)?
4. Who is the best player in your opinion based on his record stats? note: you can refer to variables point (PTS), assists, rebounds, or anything else. A combination of several variables would be nice.
5. Which team has the best average stat record of their players? Note: you can refer to points, assists, rebounds, or anything else. A combination of several variables would be nice

## Data Pre-Processing

In [1]:
#Import Library
import pandas as pd
from sklearn import preprocessing

In [2]:
#Import Dataset
df = pd.read_csv(r'D:\Personal Project\Learning on Data Science\IYKRA\4. Statistika\Fellowship-5-main\Seasons_Stats.csv')
df.columns.unique()

Index(['Unnamed: 0', 'Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'blanl', 'OWS', 'DWS', 'WS', 'WS/48', 'blank2',
       'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

Since we will only be using the 2017 data, we will filter the dataset.

In [3]:
# Filter Data 2017
nba = df[df.Year==2017]
nba.shape

(595, 53)

After filtering the data, let's have a general look on whether there are any missing values, duplicate datas, and etc. which need further cleaning.

In [4]:
# Check for general information in the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24691 entries, 0 to 24690
Data columns (total 53 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  24691 non-null  int64  
 1   Year        24624 non-null  float64
 2   Player      24624 non-null  object 
 3   Pos         24624 non-null  object 
 4   Age         24616 non-null  float64
 5   Tm          24624 non-null  object 
 6   G           24624 non-null  float64
 7   GS          18233 non-null  float64
 8   MP          24138 non-null  float64
 9   PER         24101 non-null  float64
 10  TS%         24538 non-null  float64
 11  3PAr        18839 non-null  float64
 12  FTr         24525 non-null  float64
 13  ORB%        20792 non-null  float64
 14  DRB%        20792 non-null  float64
 15  TRB%        21571 non-null  float64
 16  AST%        22555 non-null  float64
 17  STL%        20792 non-null  float64
 18  BLK%        20792 non-null  float64
 19  TOV%        19582 non-nul

From the results, we can see that there are no missing data in the dataset. However, there are some columns which is blank. Thus, these columns will be deleted.

In [5]:
# Dropping some of the blank columns.
nba = nba.drop(columns=['blanl','blank2'])
nba.shape

(595, 51)

There is a drop in the column number from 53 to 51 which means that we succesfully drop the two blank columns. 
Next, we should investigate whether there are duplicates. Since players often get transferred during one seasons, it is likely there are duplicates of player names. Thus, we should check whether there are duplicates in the dataset based on players.

In [6]:
# Duplicate Values
nba = nba.drop_duplicates((['Player']))
nba.duplicated('Player')
nba

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
24096,24096,2017.0,Alex Abrines,SG,23.0,OKC,68.0,6.0,1055.0,10.1,...,0.898,18.0,68.0,86.0,40.0,37.0,8.0,33.0,114.0,406.0
24097,24097,2017.0,Quincy Acy,PF,26.0,TOT,38.0,1.0,558.0,11.8,...,0.750,20.0,95.0,115.0,18.0,14.0,15.0,21.0,67.0,222.0
24100,24100,2017.0,Steven Adams,C,23.0,OKC,80.0,80.0,2389.0,16.5,...,0.611,282.0,333.0,615.0,86.0,88.0,78.0,146.0,195.0,905.0
24101,24101,2017.0,Arron Afflalo,SG,31.0,SAC,61.0,45.0,1580.0,9.0,...,0.892,9.0,116.0,125.0,78.0,21.0,7.0,42.0,104.0,515.0
24102,24102,2017.0,Alexis Ajinca,C,28.0,NOP,39.0,15.0,584.0,12.9,...,0.725,46.0,131.0,177.0,12.0,20.0,22.0,31.0,77.0,207.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24686,24686,2017.0,Cody Zeller,PF,24.0,CHO,62.0,58.0,1725.0,16.7,...,0.679,135.0,270.0,405.0,99.0,62.0,58.0,65.0,189.0,639.0
24687,24687,2017.0,Tyler Zeller,C,27.0,BOS,51.0,5.0,525.0,13.0,...,0.564,43.0,81.0,124.0,42.0,7.0,21.0,20.0,61.0,178.0
24688,24688,2017.0,Stephen Zimmerman,C,20.0,ORL,19.0,0.0,108.0,7.3,...,0.600,11.0,24.0,35.0,4.0,2.0,5.0,3.0,17.0,23.0
24689,24689,2017.0,Paul Zipser,SF,22.0,CHI,44.0,18.0,843.0,6.9,...,0.775,15.0,110.0,125.0,36.0,15.0,16.0,40.0,78.0,240.0


The numbers of rows decresed. Thus, we can conclude that there are duplicates in the Players Name and we have successfully drop them.

Since we have clean the data, we can now proceed to the main analysis.

# 1. Who is the youngest and oldest player in the NBA in 2017 for each team (Tm) ?

### The Oldest Player from each team:

In [7]:
# Create a new column called group_rank that rank each player at each team by age (from Oldest to Youngest)
nba['group_rank'] = nba.groupby('Tm')['Age'].rank(ascending=0, method='dense')
# Create a new table for the oldest player
nbaold = nba[nba['group_rank']==1.0][['Player','Age', 'Tm']].set_index('Tm')
nbaold.sort_values('Tm')

,Player,Age
Tm,,
ATL,Thabo Sefolosha,32.0
ATL,Gary Neal,32.0
BOS,Gerald Green,31.0
BRK,Luis Scola,36.0
CHI,Dwyane Wade,35.0
CHO,Brian Roberts,31.0
CLE,Chris Andersen,38.0
DAL,Dirk Nowitzki,38.0
DEN,Mike Miller,36.0


### The Youngest from each team:

In [8]:
# Create a new column called group_rank that rank each player at each team by age (from Youngest to Oldest)
nba['group_rank'] = nba.groupby('Tm')['Age'].rank(ascending=1, method='dense')

# Create new table for the youngest player from each team.
nbayoung = nba[nba['group_rank']==1.0][['Player','Age', 'Tm']].set_index('Tm')
nbayoung.sort_values('Tm')

,Player,Age
Tm,,
ATL,DeAndre' Bembry,22.0
ATL,Taurean Waller-Prince,22.0
BOS,Jaylen Brown,20.0
BRK,Isaiah Whitehead,21.0
CHI,Bobby Portis,21.0
CHO,Christian Wood,21.0
CLE,Kay Felder,21.0
DAL,Ben Bentil,21.0
DEN,Jamal Murray,19.0


# 2. Which player has the most minutes played (MP) in each position (Pos)?

Firstly, I would like to know what is the maximum minutes played in each position. This information could be used to crosscheck the following data.

In [9]:
age = nba.groupby('Pos').agg({'MP': ['mean', 'min', 'max']}) 
print(age)

               MP               
             mean    min     max
Pos                             
C     1129.635417    7.0  3030.0
PF    1082.113402    9.0  2803.0
PF-C   980.000000  980.0   980.0
PG    1236.406250   11.0  2947.0
SF    1406.511111   12.0  3048.0
SG    1271.056604    1.0  2796.0


After knowing the max of minutes played, let's find out who is the player:

In [10]:
nba['group_rank'] = nba.groupby('Pos')['MP'].rank(ascending=0, method='dense')
nbaMP = nba[nba['group_rank']==1.0][['Player','MP', 'Pos']].set_index('Pos')
nbaMP.sort_values('Pos')

,Player,MP
Pos,,
C,Karl-Anthony Towns,3030.0
PF,Harrison Barnes,2803.0
PF-C,Joffrey Lauvergne,980.0
PG,James Harden,2947.0
SF,Andrew Wiggins,3048.0
SG,C.J. McCollum,2796.0


# Which team has the highest average total rebound percentage (TRB%), assist percentage (AST%), steal percentage (STL%), and block percentage (BLK%)?

In [11]:
teamavg2 = nba.groupby(['Tm'])['TRB%','AST%','STL%','BLK%'].mean()

<ipython-input-11-7c9ee3b33cb9>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  teamavg2 = nba.groupby(['Tm'])['TRB%','AST%','STL%','BLK%'].mean()


In [12]:
teamavg2.nlargest(1,'TRB%')

,TRB%,AST%,STL%,BLK%
Tm,,,,
WAS,13.45,11.314286,1.535714,1.292857


In [13]:
teamavg2.nlargest(1,'AST%')

,TRB%,AST%,STL%,BLK%
Tm,,,,
DEN,10.6,15.86,1.88,0.953333


In [14]:
teamavg2.nlargest(1,'STL%')

,TRB%,AST%,STL%,BLK%
Tm,,,,
MIN,9.992857,12.664286,2.371429,1.585714


In [15]:
teamavg2.nlargest(1,'BLK%')

,TRB%,AST%,STL%,BLK%
Tm,,,,
GSW,11.426667,13.96,1.733333,2.74


# Who is the best player in your opinion based on his record stats? note: you can refer to variables point (PTS), assists, rebounds, or anything else. A combination of several variables would be nice.

To answer this question, I would like to combine a composite of:
TS% = True Shooting Percentage
TRB% = Total Rebound Percentage
AST% = Assist Percentage
BLK% = Block Percentage
STL% = Steal Percentage
This component represents the attack and defensive component from each player which contribute to winning games.


In [16]:
nba[['TS%','TRB%','AST%','BLK%','STL%']].describe()

,TS%,TRB%,AST%,BLK%,STL%
count,485.000000,486.000000,486.000000,486.000000,486.000000
mean,0.526944,10.145267,13.067695,1.664403,1.545062
std,0.089771,5.201848,9.172849,1.690613,0.923243
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.502000,6.225000,6.525000,0.500000,1.100000
50%,0.537000,8.950000,10.100000,1.200000,1.400000
75%,0.576000,13.100000,17.575000,2.400000,1.900000
max,0.799000,56.400000,57.300000,17.300000,11.100000


From the tables above, we can see that each of our variables has different min-max scale, therefore whe should try to normalize using min-max scalar. Then, calculate the composite. We put more weight on 3P% as they worth more points than other and also have more risks.

In [17]:
mms = preprocessing.MinMaxScaler()
nba[['TS%','TRB%','AST%','BLK%','STL%']] = mms.fit_transform(nba[['TS%','TRB%','AST%','BLK%','STL%']])
nba['Composite'] = nba['TS%'] + nba['TRB%'] + nba['AST%'] + nba['BLK%'] + nba['STL%']
nba[['TS%','TRB%','AST%','BLK%','STL%','Composite']].describe()

,TS%,TRB%,AST%,BLK%,STL%,Composite
count,485.000000,486.000000,486.000000,486.000000,486.000000,485.000000
mean,0.659505,0.179881,0.228058,0.096208,0.139195,1.302111
std,0.112355,0.092231,0.160085,0.097723,0.083175,0.268257
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.106383
25%,0.628285,0.110372,0.113874,0.028902,0.099099,1.153126
50%,0.672090,0.158688,0.176265,0.069364,0.126126,1.288083
75%,0.720901,0.232270,0.306719,0.138728,0.171171,1.440701
max,1.000000,1.000000,1.000000,1.000000,1.000000,2.609346


That was long, but now we can see which player is the best:

In [23]:
# Finding the best player
nba[['Player','Tm', 'Pos','Composite']].sort_values(by = ['Composite'], ascending = False).head(5)

,Player,Tm,Pos,Composite
24600,Jarnell Stokes,DEN,C,2.609346
24374,Brice Johnson,LAC,PF,2.538354
24604,Edy Tavares,TOT,C,2.264224
24654,Russell Westbrook,OKC,PG,2.255789
24306,James Harden,HOU,PG,2.106321


**Jamell Stokes** from is the Best Player by our calculation.

# Which team has the best average stat record of their players? Note: you can refer to points, assists, rebounds, or anything else. A combination of several variables would be nice?

In [19]:
# Calculate mean for eac team.
nba.groupby('Tm')[['Composite']].mean('Composite').sort_values('Composite',ascending=False).head()

,Composite
Tm,
GSW,1.473075
DEN,1.398344
SAS,1.379203
LAC,1.358393
BOS,1.348758


Using the same composite, we find **San Antonio Spurs** have the best average stat record of thei players.